In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy 
import os
# import seaborn as sns
# import scipy.stats as stats
import unittest

## Battery Dataset

In [2]:
#Upload metadata
metadata = pd.read_csv('snl_metadata_cycle_500.csv')

#Define data directory and load files, ensure the data directory and this notebook are in the same directory
data_dir = "Cycle Data"
files = os.listdir(data_dir)

#Initialize dataframe header row 
df = pd.read_csv(data_dir+"\\"+files[0])
df_output = df.head(0)

# Define cycles 
cycle = [50.0, 100.0, 150.0, 200.0, 250.0, 300.0, 350.0, 400.0, 450.0, 500.0]

  

In [3]:
#Build dataframe by adding the correct row of data pulled from "Cycle" files
#Include the output "Coulombic efficiency"
#Add the cell_id from "Cycle" files

cell_id_2 =[]

i=0                
for file in files:
    df = pd.read_csv(data_dir+"\\"+files[i])
    
    for j in range(len(cycle)):
        df_cycles = df.loc[df['Cycle_Index'] == cycle[j]]
        df_output = df_output.append(df_cycles, ignore_index = True)
        
        df_output['Coulombic_Efficiency (%)'] = df_output['Discharge_Capacity (Ah)']/df_output['Charge_Capacity (Ah)']*100
        
        cell_id_2.append(file[:-15])
        df_output['cell_id_2'] = cell_id_2
        
    i+=1     

In [4]:
df_output = pd.concat([metadata, df_output],axis=1)

In [5]:
#Define data directory and load files
#All "Time Data #" zip files need to be unzipped, create a"Time Data" directory, move all files from "Time Data #" zip folders to new "Time Data"
#Ensure the Time data directory and this notebook are in the same directory
data_dir2 = "Time Data"
files2 = os.listdir(data_dir2)

#Create new columns 
df_output['Voltages (V)'] = None 
df_output['Discharge_Capacities (Ah)'] = None 

In [6]:
#Pull array of data from "Time" files and append them into one array
voltages = []
discharge_capacities = []
i=0     
for file in files2:  
    df2 = pd.read_csv(data_dir2+"\\"+files2[i])
    
    for j in range(10):
        df_time = df2.loc[df2['Cycle_Index'] == df_output['Cycle'][j]]
        voltages.append(df_time['Voltage (V)'].values)
        discharge_capacities.append(df_time['Discharge_Capacity (Ah)'].values)
    i+=1    
    
#Populate the newly created columns with each cell having an array of data
for i in range(len(df_output)):
    df_output['Voltages (V)'][i] = voltages[i]
    df_output['Discharge_Capacities (Ah)'][i] = discharge_capacities[i]

<ipython-input-6-271e406e8bb8>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output['Voltages (V)'][i] = voltages[i]
<ipython-input-6-271e406e8bb8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output['Discharge_Capacities (Ah)'][i] = discharge_capacities[i]


In [7]:
#Check to make sure data pulled from "Cycle" files match the correct metadata line
for i in range(len(df_output)):
    if df_output['cell_id'][i] == df_output['cell_id_2'][i]:
        pass
    else:
        print('not a match', df_output['cell_id'][i],df_output['cell_id_2'][i])
        
for i in range(len(df_output)):
    if df_output['Cycle'][i] == df_output['Cycle_Index'][i]:
        pass
    else:
        print('not a match', df_output['Cycle'][i],df_output['Cycle_Index'][i])

In [8]:
#Load full Battery dataset
df_battery_full = df_output
df_battery_full = df_battery_full.drop('cell_id_2', 1)
df_battery_full = df_battery_full.drop('Cycle_Index', 1)

In [9]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# df_battery_full

## Battery Dataset Filtered

In [10]:
#Create a column that will indicated replicated experiments 
df_battery_full['rep'] = None 

#Populate the column, replicates will have a value of 1
for i in range(len(df_battery_full)):
    if df_battery_full.cell_id.values[i][-1] == 'a':
        df_battery_full['rep'][i] = 0
    if df_battery_full.cell_id.values[i][-1] != 'a':
        df_battery_full['rep'][i] = 1

#Re

<ipython-input-10-87105f65baed>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_battery_full['rep'][i] = 0
<ipython-input-10-87105f65baed>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_battery_full['rep'][i] = 1


In [11]:
#Drop any row of replicated experiment data and load filtered Battery Dataset
df_battery = df_battery_full[df_battery_full['rep'] < 0.5]
df_battery = df_battery.drop('rep', 1)
df_battery = df_battery.reset_index(drop=True)

In [12]:
df_battery


,cell_id,study,Cycles,anode,cathode,electrolyte,Cycle,temperature,min_soc,max_soc,...,Max_Current (A),Min_Voltage (V),Max_Voltage (V),Charge_Capacity (Ah),Discharge_Capacity (Ah),Charge_Energy (Wh),Discharge_Energy (Wh),Coulombic_Efficiency (%),Voltages (V),Discharge_Capacities (Ah)
0,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",graphite,LFP,A123,50,15,0,100,...,0.55,1.998,3.6,1.036,1.033,3.495,3.213,99.710425,"[2.985, 3.123, 3.205, 3.262, 3.287, 3.294, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",graphite,LFP,A123,100,15,0,100,...,0.55,1.996,3.6,1.035,1.032,3.492,3.212,99.710145,"[2.985, 3.123, 3.205, 3.262, 3.286, 3.294, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",graphite,LFP,A123,150,15,0,100,...,0.55,1.998,3.6,1.034,1.031,3.488,3.210,99.709865,"[2.985, 3.123, 3.205, 3.262, 3.286, 3.293, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",graphite,LFP,A123,200,15,0,100,...,0.55,1.994,3.6,1.033,1.030,3.490,3.193,99.709584,"[2.992, 3.129, 3.212, 3.268, 3.292, 3.299, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",graphite,LFP,A123,250,15,0,100,...,0.55,1.998,3.6,1.031,1.028,3.495,3.169,99.709020,"[3.002, 3.139, 3.222, 3.278, 3.302, 3.309, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,graphite,NMC,LGC,300,35,0,100,...,1.50,1.996,4.2,2.529,2.527,9.814,8.388,99.920917,"[1.996, 3.252, 3.413, 3.493, 3.541, 3.561, 3.5...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
316,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,graphite,NMC,LGC,350,35,0,100,...,1.50,1.994,4.2,2.518,2.506,9.769,8.332,99.523431,"[1.994, 3.244, 3.41, 3.493, 3.54, 3.56, 3.577,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
317,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,graphite,NMC,LGC,400,35,0,100,...,1.50,1.995,4.2,2.484,2.482,9.650,8.197,99.919485,"[1.995, 3.261, 3.423, 3.504, 3.549, 3.568, 3.5...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
318,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,graphite,NMC,LGC,450,35,0,100,...,1.50,1.995,4.2,2.477,2.478,9.617,8.208,100.040371,"[1.995, 3.25, 3.417, 3.501, 3.547, 3.566, 3.58...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# Turn string inputs into numerical values

In [13]:
for i in range(len(df_battery)):
    if df_battery.at[i,'anode'] == 'graphite':
        df_battery.at[i,'anode'] = 0
    if df_battery.at[i,'cathode'] == 'LFP':
        df_battery.at[i,'cathode'] = 0
    if df_battery.at[i,'cathode'] == 'NCA':
        df_battery.at[i,'cathode'] = 1
    if df_battery.at[i,'cathode'] == 'NMC':
        df_battery.at[i,'cathode'] = 2
    if df_battery.at[i,'electrolyte'] == 'A123':
        df_battery.at[i,'electrolyte'] = 0
    if df_battery.at[i,'electrolyte'] == 'Pan':
        df_battery.at[i,'electrolyte'] = 1
    if df_battery.at[i,'electrolyte'] == 'LGC':
        df_battery.at[i,'electrolyte'] = 2

In [14]:
df_battery

,cell_id,study,Cycles,anode,cathode,electrolyte,Cycle,temperature,min_soc,max_soc,...,Max_Current (A),Min_Voltage (V),Max_Voltage (V),Charge_Capacity (Ah),Discharge_Capacity (Ah),Charge_Energy (Wh),Discharge_Energy (Wh),Coulombic_Efficiency (%),Voltages (V),Discharge_Capacities (Ah)
0,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",0,0,0,50,15,0,100,...,0.55,1.998,3.6,1.036,1.033,3.495,3.213,99.710425,"[2.985, 3.123, 3.205, 3.262, 3.287, 3.294, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",0,0,0,100,15,0,100,...,0.55,1.996,3.6,1.035,1.032,3.492,3.212,99.710145,"[2.985, 3.123, 3.205, 3.262, 3.286, 3.294, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",0,0,0,150,15,0,100,...,0.55,1.998,3.6,1.034,1.031,3.488,3.210,99.709865,"[2.985, 3.123, 3.205, 3.262, 3.286, 3.293, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",0,0,0,200,15,0,100,...,0.55,1.994,3.6,1.033,1.030,3.490,3.193,99.709584,"[2.992, 3.129, 3.212, 3.268, 3.292, 3.299, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",0,0,0,250,15,0,100,...,0.55,1.998,3.6,1.031,1.028,3.495,3.169,99.709020,"[3.002, 3.139, 3.222, 3.278, 3.302, 3.309, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,0,2,2,300,35,0,100,...,1.50,1.996,4.2,2.529,2.527,9.814,8.388,99.920917,"[1.996, 3.252, 3.413, 3.493, 3.541, 3.561, 3.5...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
316,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,0,2,2,350,35,0,100,...,1.50,1.994,4.2,2.518,2.506,9.769,8.332,99.523431,"[1.994, 3.244, 3.41, 3.493, 3.54, 3.56, 3.577,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
317,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,0,2,2,400,35,0,100,...,1.50,1.995,4.2,2.484,2.482,9.650,8.197,99.919485,"[1.995, 3.261, 3.423, 3.504, 3.549, 3.568, 3.5...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
318,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,0,2,2,450,35,0,100,...,1.50,1.995,4.2,2.477,2.478,9.617,8.208,100.040371,"[1.995, 3.25, 3.417, 3.501, 3.547, 3.566, 3.58...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [15]:
%store df_battery
df_battery

Stored 'df_battery' (DataFrame)


,cell_id,study,Cycles,anode,cathode,electrolyte,Cycle,temperature,min_soc,max_soc,...,Max_Current (A),Min_Voltage (V),Max_Voltage (V),Charge_Capacity (Ah),Discharge_Capacity (Ah),Charge_Energy (Wh),Discharge_Energy (Wh),Coulombic_Efficiency (%),Voltages (V),Discharge_Capacities (Ah)
0,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",0,0,0,50,15,0,100,...,0.55,1.998,3.6,1.036,1.033,3.495,3.213,99.710425,"[2.985, 3.123, 3.205, 3.262, 3.287, 3.294, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",0,0,0,100,15,0,100,...,0.55,1.996,3.6,1.035,1.032,3.492,3.212,99.710145,"[2.985, 3.123, 3.205, 3.262, 3.286, 3.294, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",0,0,0,150,15,0,100,...,0.55,1.998,3.6,1.034,1.031,3.488,3.210,99.709865,"[2.985, 3.123, 3.205, 3.262, 3.286, 3.293, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",0,0,0,200,15,0,100,...,0.55,1.994,3.6,1.033,1.030,3.490,3.193,99.709584,"[2.992, 3.129, 3.212, 3.268, 3.292, 3.299, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,SNL_18650_LFP_15C_0-100_0.5-1C_a,snl,"4,551.00",0,0,0,250,15,0,100,...,0.55,1.998,3.6,1.031,1.028,3.495,3.169,99.709020,"[3.002, 3.139, 3.222, 3.278, 3.302, 3.309, 3.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,0,2,2,300,35,0,100,...,1.50,1.996,4.2,2.529,2.527,9.814,8.388,99.920917,"[1.996, 3.252, 3.413, 3.493, 3.541, 3.561, 3.5...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
316,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,0,2,2,350,35,0,100,...,1.50,1.994,4.2,2.518,2.506,9.769,8.332,99.523431,"[1.994, 3.244, 3.41, 3.493, 3.54, 3.56, 3.577,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
317,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,0,2,2,400,35,0,100,...,1.50,1.995,4.2,2.484,2.482,9.650,8.197,99.919485,"[1.995, 3.261, 3.423, 3.504, 3.549, 3.568, 3.5...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
318,SNL_18650_NMC_35C_0-100_0.5-2C_a,snl,786,0,2,2,450,35,0,100,...,1.50,1.995,4.2,2.477,2.478,9.617,8.208,100.040371,"[1.995, 3.25, 3.417, 3.501, 3.547, 3.566, 3.58...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
